In [54]:
'''
匯入套件
'''
# 操作 browser 的 API
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

# ChromeDriver 的下載管理工具
from webdriver_manager.chrome import ChromeDriverManager

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver import Keys, ActionChains
# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import os

# 子處理程序，用來取代 os.system 的功能
import subprocess

# 下載檔案的工具
import wget

In [103]:
# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome()

# 建立儲存圖片、影片的資料夾
folderPath = 'music'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

# 放置爬取的資料
listData = []

The chromedriver version (121.0.6167.184) detected in PATH at C:\Users\USER\python_web_scraping-master\chromedriver.exe might not be compatible with the detected chrome version (122.0.6261.69); currently, chromedriver 122.0.6261.69 is recommended for chrome 122.*, so it is advised to delete the driver in PATH and retry


In [110]:
'''
以 function 名稱，作為爬蟲流程
'''
#visit
def visit():
    driver.get('https://www.youtube.com/')

def search():
   
    textInput = driver.find_element(
        By.CSS_SELECTOR,
        'input#search'
    )
    textInput.send_keys("james smiths")
    #
    sleep(1)
    # 模拟按下回车键，提交内容
    textInput.send_keys(Keys.ENTER)

    sleep(3)
   
def filterFunc():
    try:
        # 等待篩選元素出現
        WebDriverWait(driver,5).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR,'ytd-toggle-button-renderer.style-scope.ytd-search-sub-menu-renderer')
            )
        )
    
        #按下篩選元素，使項目浮現
        driver.find_element(
                By.CSS_SELECTOR, 
                "ytd-button-renderer.style-scope.ytd-search-header-renderer"
            ).click()
    
        # 等待一下
        sleep(2)
    
        driver.find_elements(
            By.CSS_SELECTOR,
            'a#endpoint.yt-simple-endpoint.style-scope.ytd-search-filter-renderer'
            )[9].click()
    
        sleep(3)

    except TimeoutException:
        print('等待逾時')
    

def scroll():
    '''
    innerHeight => 瀏覽器內部的高度
    offset => 當前捲動的量(高度)
    count => 累計無效滾動次數
    limit => 最大無效滾動次數
    '''
    innerHeight = 0
    offset = 0
    count = 0
    limit = 3

    while count<=limit:
        offset = driver.execute_script(
            'return document.documentElement.scrollHeight'
        )

    # 捲軸往下滑動
        driver.execute_script(f'''
          window.scrollTo({{
          top:{offset},
          behavior:"smooth"
          }})  
    ''')
        sleep(3)
    # 透過執行 js 語法來取得捲動後的當前總高度
        innerHeight = driver.execute_script(
            'return document.documentElement.scrollHeight;'
        );
         # 經過計算，如果滾動距離(offset)大於等於視窗內部總高度(innerHeight)，代表已經到底了
        if offset == innerHeight:
            count += 1
            
        if offset >= 600:
                break

def parse():
    global listData
    listData.clear()

    elements = driver.find_elements(
        By.CSS_SELECTOR,
        'ytd-video-renderer.style-scope.ytd-item-section-renderer'
    )
    for elm in elements:
        #name
        print('*'*30)
        name = elm.find_element(
            By.CSS_SELECTOR,
            'a#video-title'
        )
        name = name.get_attribute('innerText')
        print(name)

        #href
        s_link = elm.find_element(
            By.CSS_SELECTOR,
            'a#video-title'
        )
        link = s_link.get_attribute('href')
        print(link)

        listData.append(
            {
                "title":name,
                "link":link
            }
        )

# 將 list 存成 json
def saveJs():
    with open(f"{folderPath}/music.json",'w',encoding='utf-8') as file:
        file.write(json.dumps(listData ,ensure_ascii=False, indent=4) )

def close():
    driver.quit()

In [111]:
if __name__ == "__main__" :
    visit()
    search()
    song()
    scroll()
    parse()
    saveJs()

IndexError: list index out of range

In [95]:
close()

In [96]:
# 下載
def download():
    if not os.path.exists('./yt-dlp.exe'):
        print('下載yt-dlp')
        wget.download('https://github.com/yt-dlp/yt-dlp/releases/latest/download/yt-dlp.exe', './yt-dlp.exe')
    # 開啟 json 檔案
    with open(f'{folderPath}/music.json',"r",encoding='utf-8') as file:
        #取得 json 字串
        strJson = file.read()
      # 將 json 轉成 list (裡面是 dict 集合)
        listResult = json.loads(strJson)
        
        # 下載所有檔案
        for index,obj in enumerate(listResult):
            if index == 3:
                break
            print(f"正在下載:{obj['link']}")
        # 定義指令
        cmd = [
            './yt-dlp.exe',
            obj['link'],
            '-f','b[ext=mp3]]',
            '-o',f'{folderPath}/%(title).%(ext).'
        ]
        # 執行指令，並取得回傳結果 (subprocess 物件)
        obj_sp = subprocess.run(cmd)
         # 判斷指令行是否正常 (returncode == 0 代表正常)
        if obj_sp.returncode == 0:
            print('下載成功!!')
        else:
            print('下載失敗…')

'''另行下載'''
download()

正在下載:https://music.youtube.com/watch?v=LvskYHLPiKI
正在下載:https://music.youtube.com/watch?v=77fho_pCNFA
正在下載:https://music.youtube.com/watch?v=kAo4sWbZP_A
下載失敗…


In [97]:
obj

NameError: name 'obj' is not defined

In [ ]:
def login():
    textInput = driver.find_element(
         By.CSS_SELECTOR,
        'input[ type=email]'
    )
    textInput.send_keys("maggie789maggie77889@gmail.com")

    driver.find_elements(
        By.CSS_SELECTOR,
        'button[type =button]'
    )[3].click()
    #
    sleep(2)

    
    textInput2 = driver.find_element(
        By.CSS_SELECTOR,
        'input[type="password"]'
    )
    textInput2.send_keys("Maggie0119")